# Overview of peptigate results on TSA tick salivary gland transcriptomes

This notebook provides a very brief overview of the peptides predicted in tick salivary gland transcriptomes by the peptigate pipeline.
It joins output files and slices them in various ways to count the number and types of peptides predicted by peptigate.

## Notebook setup

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
setwd("..")

## Read in and tally peptigate results

In [3]:
peptigate_predictions <- Sys.glob("outputs/tsa_tick_sg_transcriptomes/*/predictions/peptide_predictions.tsv") %>%
  map_dfr(read_tsv, show_col_types = F)

In [28]:
peptigate_predictions_distinct <- peptigate_predictions %>%
  select(-peptide_id) %>%
  select(-nlpprecursor_class_score, -nlpprecursor_cleavage_score, -nucleotide_sequence,
         -start, -end, -peptide_class) %>%
  distinct()

In [29]:
peptigate_predictions_distinct %>%
  group_by(peptide_type) %>%
  tally()

peptide_type,n
<chr>,<int>
cleavage,17928
sORF,208610


In [30]:
peptigate_predictions_distinct %>%
  group_by(prediction_tool) %>%
  tally()

prediction_tool,n
<chr>,<int>
deeppeptide,14754
nlpprecursor,3174
plmutils,208610


In [31]:
nrow(peptigate_predictions_distinct)

[1] 226538

In [32]:
length(unique(peptigate_predictions_distinct$protein_sequence))

[1] 226532

In [18]:
peptigate_annotations_distinct <- Sys.glob("outputs/tsa_tick_sg_transcriptomes/*/predictions/peptide_annotations.tsv") %>%
  map_dfr(read_tsv, show_col_types = F) %>%
  group_by(sequence) %>%
  slice_head(n = 1)

In [21]:
peptigate_all_distinct <- left_join(peptigate_predictions_distinct, peptigate_annotations_distinct,
                                    by = c("protein_sequence" = "sequence"))

In [24]:
colnames(peptigate_all_distinct)

[1] "peptide_type"                     "plmutils_class_probability"      
 [3] "protein_sequence"                 "peptide_id"                      
 [5] "AB"                               "ACE"                             
 [7] "ACP"                              "AF"                              
 [9] "AMAP"                             "AMP"                             
[11] "AOX"                              "APP"                             
[13] "AV"                               "BBP"                             
[15] "DPPIV"                            "MRSA"                            
[17] "Neuro"                            "QS"                              
[19] "TOX"                              "TTCA"                            
[21] "aliphatic_index"                  "boman_index"                     
[23] "charge"                           "hydrophobicity"                  
[25] "instability_index"                "isoelectric_point"               
[27] "molecular_weight"                 "pd1_residue_volume"              
[29] "pd2_hydrophilicity"               "z1_lipophilicity"                
[31] "z2_steric_bulk_or_polarizability" "z3_polarity_or_charge"           
[33] "z4_electronegativity_etc"         "z5_electronegativity_etc"        
[35] "deepsig_combined"                 "peptipedia_blast_sseqid"         
[37] "peptipedia_blast_full_sseq"       "peptipedia_blast_pident"         
[39] "peptipedia_blast_length"          "peptipedia_blast_qlen"           
[41] "peptipedia_blast_slen"            "peptipedia_blast_mismatch"       
[43] "peptipedia_blast_gapopen"         "peptipedia_blast_qstart"         
[45] "peptipedia_blast_qend"            "peptipedia_blast_sstart"         
[47] "peptipedia_blast_send"            "peptipedia_blast_evalue"         
[49] "peptipedia_blast_bitscore"        "peptipedia_num_hits"

In [25]:
# if peptide was predicted by multiple tools, randomly select first sequence and tally how many peptides each tool predicted
peptigate_predictions %>%
  group_by(protein_sequence) %>%
  slice_head(n = 1) %>%
  ungroup() %>%
  group_by(prediction_tool) %>%
  tally()

prediction_tool,n
<chr>,<int>
deeppeptide,14749
nlpprecursor,3174
plmutils,208609


In [26]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/taylorreiter/miniconda3/envs/tidyjupyter/lib/libopenblasp-r0.3.26.dylib;  LAPACK version 3.12.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.3 forcats_1.0.0   stringr_1.5.1   dplyr_1.1.4    
 [5] purrr_1.0.2     readr_2.1.5     tidyr_1.3.1     tibble_3.2.1   
 [9] ggplot2_3.5.0   tidyverse_2.0.0

loaded via a namespace (and not attached):
 [1] bit_4.0.5        gtable_0.3.4     jsonlite_1.8.8   compiler_4.3.3  
 [5] crayon_1.5.2     tidyselect_1.2.0 IRdisplay_1.1    parallel_4.3.3  
 [9] scales_1.3.0     uuid_1.2-0       fastmap_1.1.1    IRkernel_1.3.2  
[13] R6_2.5.1         generics_0.1.3   munsell_0.5.1  